In [ ]:
from tqdm.auto import tqdm
from time import sleep

for i in (ep_bar:= tqdm(range(4), desc='1st loop')):
    ep_bar.set_description(f'Epoch {i}')
    for j in tqdm(range(5), desc='2nd loop', leave= False):
        for k in tqdm(range(50), desc='3rd loop', leave=False):
            sleep(0.01)

In [6]:
for i, j in enumerate([1,2,3,4,5,6,7,8]):
    print(i,j)

0 1
1 2
2 3
3 4
4 5
5 6
6 7
7 8


In [1]:
import torch
from spoof_gen.models.backbone.upblock import build_decoder_unet
from spoof_gen.models.backbone.mobilenet import mobilenet
from spoof_gen.models.backbone.resnet import resnet
from spoof_gen.models.backbone.revnet import revnet
from spoof_gen.models.generator import Encoder, Decoder, Generator
from spoof_gen.models.conditional_generator import ConditionGenerator
# decoder = build_decoder_unet(skip_z= True)

N = 1
x = torch.rand((N,3,256,256)).cuda()
z = torch.rand((N,4,32,32)).cuda()

In [2]:
# decoder = Decoder().cuda()

In [3]:
con_gen = ConditionGenerator().cuda()
encoder = Encoder('resnet-50').cuda()
decoder = Decoder('revnet-50').cuda()
gen = Generator(encoder= encoder, decoder= decoder, con_gen= con_gen)

In [ ]:
for i in range(100):
    gen_resp = gen(x)

In [4]:
# gen_resp.shape

In [2]:
!pip install git+https://github.com/minh-nguyenhoang/SpoofGen

  Cloning https://github.com/minh-nguyenhoang/SpoofGen to c:\users\admin\appdata\local\temp\pip-req-build-1gkdw1ia
  Resolved https://github.com/minh-nguyenhoang/SpoofGen to commit f9acce48ddc87c4b3df0feca0d67d7e426df71be
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for spoof-gen: filename=spoof_gen-0.0.1-py3-none-any.whl size=3987 sha256=b944e7623c443e58ad16bd7deb63ed8a3ebe68613ea800b34d2ddcd8d52fa452
  Stored in directory: C:\Users\ADMIN\AppData\Local\Temp\pip-ephem-wheel-cache-vgkqz61m\wheels\85\0f\2a\a314b5a9df81da73fc8ad53aab82c6bc91353e324084985683
Successfully built spoof-gen


  Running command git clone --filter=blob:none --quiet https://github.com/minh-nguyenhoang/SpoofGen 'C:\Users\ADMIN\AppData\Local\Temp\pip-req-build-1gkdw1ia'

[notice] A new release of pip is available: 23.1 -> 23.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import spoof_gen

ModuleNotFoundError: No module named 'spoof_gen'